Indexing dataset
================

In [1]:
from pathlib import Path
import sys
debug_local = True#False
local = (Path("..") / "indexpaper").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  print("extending pathes with local indexpaper")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local indexpaper
['..', '/home/antonkulaga/sources/indexpaper/notebooks', '/home/antonkulaga/sources/indexpaper', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python310.zip', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/lib-dynload', '', '/home/antonkulaga/.local/lib/python3.10/site-packages', '/home/antonkulaga/micromamba/envs/indexpaper/lib/python3.10/site-packages']


In [2]:
import polars as pl
from indexpaper import index

from pycomfort.files import *
from typing import List
from pathlib import Path
from polars import Config
Config.set_fmt_str_lengths(5000)

/home/antonkulaga/.local/lib/python3.10/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Callable[[list], list] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


polars.config.Config

In [3]:
from pycomfort.config import load_environment_keys
from indexpaper.paperset import Paperset
from indexpaper.resolvers import *

chunk_size = 512
embeddings, splitter = resolve_embedding_splitter(EmbeddingType.HuggingFace, EmbeddingModels.biolinkbert.value, chunk_size=chunk_size)
from indexpaper.indexing import init_qdrant
load_environment_keys(usecwd=True)
api_key = os.getenv("QDRANT_KEY")
db = init_qdrant("biolinkbert_large_512_tacutu_papers", 
                 path_or_url="https://62d4a96e-2b91-4ab8-a4dd-a91e626d874a.europe-west3-0.gcp.cloud.qdrant.io:6333/", 
                 embeddings=embeddings, always_recreate=False,
                 api_key=api_key
                 )

No sentence-transformers model found with name /home/antonkulaga/.cache/torch/sentence_transformers/michiyasunaga_BioLinkBERT-large. Creating a new one with MEAN pooling.
2023-10-03 14:14:15.884 | INFO     | indexpaper.indexing:init_qdrant:78 - initializing quadrant database at https://62d4a96e-2b91-4ab8-a4dd-a91e626d874a.europe-west3-0.gcp.cloud.qdrant.io:6333/


/home/antonkulaga/sources/indexpaper/.env
environment found at /home/antonkulaga/sources/indexpaper/.env


In [4]:
col = db.client.get_collection("biolinkbert_large_512_tacutu_papers")
col

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

# Paperset #

In [5]:
from qdrant_client.http.models import *
from typing import Union
#TODO: fix it does not work!
db.client.search("biolinkbert_large_512_tacutu_papers", None,
    Filter(
    must=[
        FieldCondition(
            key='doi',
            match=MatchValue(value = "10.1186/s13059-020-01990-9")
        )
    ]
))

ValidationError: 1 validation error for SearchRequest
vector
  none is not an allowed value (type=type_error.none.not_allowed)

In [6]:
moskalev = Paperset("longevity-genie/moskalev_papers", splitter=splitter)
moskalev.lazy_frame.collect().head(1)

corpusid,content_source_oainfo_openaccessurl,updated,externalids_doi,externalids_pubmed,annotations_abstract,annotations_author,annotations_title,annotations_paragraph
i64,str,str,str,str,list[str],list[str],list[str],list[str]
259168060,null,"""2023-07-04T07:35:35Z""","""10.3389/fnagi.2023.1179988""","""37396658""","[""Song W (2023) The glymphatic system: a new perspective on brain diseases.The glymphatic system is a brain-wide perivascular pathway driven by aquaporin-4 on the endfeet of astrocytes, which can deliver nutrients and active substances to the brain parenchyma through periarterial cerebrospinal fluid (CSF) influx pathway and remove metabolic wastes through perivenous clearance routes. This paper summarizes the composition, overall fluid flow, solute transport, related diseases, affecting factors, and preclinical research methods of the glymphatic system. In doing so, we aim to provide direction and reference for more relevant researchers in the future.""]","[""Alexey Moskalev "", ""Claudio Nicoletti "", … "" University of Pittsburgh United States ""]","[""OPEN ACCESS EDITED BY The glymphatic system: a new perspective on brain diseases"", ""OPEN ACCESS EDITED BY The glymphatic system: a new perspective on brain diseases""]","[""It is traditionally believed that the lymphatic system doesn't exist in the central nervous system (Nedergaard and Goldman, 2016). Thus, cell debris, potential neurotoxic proteins, and other metabolites with large molecular weight are considered to be removed in a different clearance pathway than brain vasculature. In 2012, the Nedergaard (Iliff et al., 2012) group found that cerebrospinal fluid (CSF) can enter brain parenchyma and exchange with brain interstitial fluid (ISF) in the presence of aquaporin-4 (AQP4) on astrocytes. Likewise, when the mixed fluid was drained from the brain, Amyloid-β (Aβ) was transported along with the outflow. Since the function of this ""drainage"" pathway is similar to that of the lymphatic system and supported by astrocytes, it was named the glymphatic system sooner after glia (Jessen et al., 2015)."", ""Since last decade, many researchers in the field of neurology, neurodegenerative diseases and physiology have aimed to study and develop the glymphatic system, providing a brandnew perspective for us to understand brain diseases: the overall fluid flow of the brain rather than a specific lesion or structure. Herein, we summarized the components of the glymphatic system, the fluid circulation mode within this system, how pathogenic solutes are transported in certain diseases, affecting factors of its function, and by what means we can study the glymphatic system. All above may provide direction and reference for brain diseases and medical researchers."", … ""This work was supported by the Funds of the scientific and technological innovation project of the Chinese Academy of Traditional Chinese Medicine CI2021A04614.""]"


In [8]:
col = db.client.get_collections()
col

CollectionsResponse(collections=[CollectionDescription(name='biolinkbert_large_512_tacutu_papers')])

Indexing
========
To avoid memory overflow we index everything by slices

In [9]:
moskalev.index_by_slices(10, db, 0)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyboardInterrupt: 

In [9]:
# Testing index


In [ ]:
db.similarity_search("IKK-β activation ")[0].page_content

In [17]:
db.similarity_search("IKK-β activation ")[0].page_content

'proteins: p65 (RelA), RelB, c-Rel, p105/p50 (NF-κB1), and p100/52 (NF-κB2), which associate with each other to form distinct active NF-κB dimers. In the cytosol, NF-κB dimers in inactive form anchored by IκB are found. There are two ways to activate the NF-kB-induced gene transcription. These are triggered by cytokines TNFα and IL-1 (canonical signaling) or antigen receptors CD40 and BAFF (non-canonical/alternative signaling). There are the activation of IKK complex (IKKα, IKKβ, and IKKγ) and the phosphorylation of IκB proteins, which are in interaction with'